<h1 style="color:rgb(17, 116, 155);text-align:left;font-size:250%;font-family:verdana;text-decoration:underline;"> 
    Advanced topics - Final Project - Part 1: Transfer Learning</h1>

In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
# Import required libraries

import torch
import torch.nn as nn
import torch.optim as optim
import torchinfo
import torchvision
import torchvision.models as models
from torch import nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from scipy.io import loadmat
import matplotlib.pyplot as plt
import random
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import datetime
import time
import numpy as np 
import pandas as pd 

In [2]:
class CarsDataset(Dataset):
    def __init__(self, annotations, img_dir, transform=None):
        self.img_paths = [os.path.join(img_dir, sample[-1][0]) for sample in annotations[0]]
        self.labels = [sample[-2][0][0] - 1 for sample in annotations[0]]
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        image = Image.open(self.img_paths[idx]).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)
        
        return image, label

cars_dataset = torch.load("cars196_dataset.pt", weights_only=False)

In [3]:
# הצגת גודל הדאטהסט
print(f"מספר דוגמאות בדאטהסט: {len(cars_dataset)}")


מספר דוגמאות בדאטהסט: 16185


In [4]:
class CarsDatasetWrapper(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __len__(self):
        return len(self.subset)

    def __getitem__(self, idx):
        image, label = self.subset[idx]  # שליפת נתונים מה-Subset
        if self.transform:
            if isinstance(image, torch.Tensor):  # **בדיקה אם התמונה כבר טנסור**
                pass  # אם זה כבר טנסור, לא צריך להפעיל שוב את `ToTensor()`
            else:
                image = self.transform(image)  # **הוספת האוגמנטציה**
        return image, label

In [5]:
# הגדרת טרנספורמציות לתמונות
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # שינוי גודל כל התמונות ל-224x224 פיקסלים
    transforms.RandomHorizontalFlip(p=0.5),  # היפוך אופקי של התמונה בהסתברות של 50%
    transforms.RandomRotation(15),  # סיבוב אקראי של התמונה עד 15 מעלות לכל כיוון
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),  # שינוי אקראי של הבהירות, הניגודיות, הרווייה והגוון של התמונה
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2)), # תזוזה אקראית של התמונה עד 20% מהגודל שלה (לרוחב ולאורך)
    transforms.RandomPerspective(distortion_scale=0.3, p=0.5),  # עיוות פרספקטיבה בהסתברות של 50%, עם עיוות מקסימלי של 30%
    transforms.RandomResizedCrop(224, scale=(0.7, 1.0)),  # חיתוך אקראי מהתמונה תוך שמירה על יחס גובה-רוחב, עם קנה מידה בין 70% ל-100% מהגודל המקורי
    transforms.ToTensor(),  # המרת התמונה ל- **טנסור של PyTorch** (C, H, W) עם טווח ערכים בין 0 ל-1
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # נרמול התמונה לערכים עם ממוצע וסטיית תקן כמו של **ImageNet**, לשיפור האימון
])


test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

# חלוקת הדאטה לאימון ולמבחן
dataset_size = len(cars_dataset)
train_size = int(0.7 * dataset_size)  # 70% מהדאטה
test_size = dataset_size - train_size  # 30% הנותרים

train_indices, test_indices = train_test_split(range(dataset_size), train_size=train_size, test_size=test_size, random_state=42)

# יצירת סטים עם אוגמנטציה
train_dataset = Subset(cars_dataset, train_indices)
test_dataset = Subset(cars_dataset, test_indices)

# עטיפת ה-Subset עם הטרנספורמציות
train_dataset = CarsDatasetWrapper(train_dataset, transform=train_transform)
test_dataset = CarsDatasetWrapper(test_dataset, transform=test_transform)

print(f"Training samples: {len(train_dataset)}, Testing samples: {len(test_dataset)}")

# יצירת DataLoader עם סטים מעודכנים
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Training samples: 11329, Testing samples: 4856


In [7]:
from torchvision.models import ResNet50_Weights

base_model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)


In [8]:
class CustomMobileNet(nn.Module):
    def __init__(self, base_model, num_classes=196):
        super(CustomMobileNet, self).__init__()
        self.base = nn.Sequential(*list(base_model.children())[:-2])  # שמירה על כל השכבות של ResNet
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)  # Pooling
        self.dropout1 = nn.Dropout(0.5)  # Dropout ראשון
        self.fc1 = nn.Linear(2048, 512)  # מעבר דרך שכבה פנימית קטנה יותר
        self.relu = nn.ReLU()  # פונקציה לא לינארית
        self.dropout2 = nn.Dropout(0.5)  # Dropout שני
        self.fc2 = nn.Linear(512, num_classes)  # השכבה הסופית ל-196 קטגוריות

    def forward(self, x):
        x = self.base(x)
        x = self.global_avg_pool(x)
        x = torch.flatten(x, 1)  # Flatten לפני ה-FC
        x = self.dropout1(x)  # Dropout ראשון
        x = self.fc1(x)  # Fully Connected 1
        x = self.relu(x)  # פונקציה לא לינארית
        x = self.dropout2(x)  # Dropout שני
        x = self.fc2(x)  # Fully Connected אחרון
        return x

# יצירת המודל עם 196 מחלקות
model = CustomMobileNet(base_model, num_classes=196)


In [9]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model.to(device)

CustomMobileNet(
  (base): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d

In [10]:
dummy_input = torch.randn(1, 3, 224, 224)  # תמונה מדומה
output = model(dummy_input)
print("Output shape:", output.shape)  # צריך להיות torch.Size([1, 196])


Output shape: torch.Size([1, 196])


In [11]:
from torchinfo import summary

summary(model, input_size=(1, 3, 224, 224), col_names=["input_size", "output_size", "num_params", "trainable"])


Layer (type:depth-idx)                        Input Shape               Output Shape              Param #                   Trainable
CustomMobileNet                               [1, 3, 224, 224]          [1, 196]                  --                        True
├─Sequential: 1-1                             [1, 3, 224, 224]          [1, 2048, 7, 7]           --                        True
│    └─Conv2d: 2-1                            [1, 3, 224, 224]          [1, 64, 112, 112]         9,408                     True
│    └─BatchNorm2d: 2-2                       [1, 64, 112, 112]         [1, 64, 112, 112]         128                       True
│    └─ReLU: 2-3                              [1, 64, 112, 112]         [1, 64, 112, 112]         --                        --
│    └─MaxPool2d: 2-4                         [1, 64, 112, 112]         [1, 64, 56, 56]           --                        --
│    └─Sequential: 2-5                        [1, 64, 56, 56]           [1, 256, 56, 56]        

In [12]:
print(f"Train Loader Size: {len(train_loader.dataset)}")
print(f"Test Loader Size: {len(test_loader.dataset)}")


Train Loader Size: 11329
Test Loader Size: 4856


In [13]:
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=1e-4, nesterov=True)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold=0.9)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

In [14]:
def train_epoch(model, train_loader, criterion, optimizer, scheduler):
    model.train()
    running_loss = 0
    
    print("Starting Training Epoch...")

    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        target = target.to(device).long()
        outputs = model(data)
        loss = criterion(outputs, target)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()

    running_loss /= len(train_loader)
    
    print('Train Loss:', running_loss)
    return running_loss


def test_model(model, test_loader, criterion):
    with torch.no_grad():
        model.eval()
        running_loss = 0
        total_predictions = 0
        correct_predictions = 0

        for batch_idx, (data, target) in enumerate(test_loader):
            data = data.to(device)
            target = target.to(device).long()
            outputs = model(data)  # forward step
            _, predicted = torch.max(outputs.data, 1)
            total_predictions += target.size(0)
            correct_predictions += (predicted == target).sum().item()
            
            loss = criterion(outputs, target)  # loss
            running_loss += loss.item()
            
        running_loss /= len(test_loader)
        acc = (correct_predictions / total_predictions) * 100.0
        print('Test Loss:', running_loss, 'Accuracy:', acc, '%')
        return running_loss, acc

In [15]:
layers_to_train = 10
for param in model.parameters():
    param.requires_grad = False  # מקפיא את כל השכבות

for param in list(model.base.parameters())[-layers_to_train:]:  
    param.requires_grad = True  # פותח השכבות האחרונות לאימון

In [16]:
n_epochs = 5
Train_loss = []
Test_loss = []
Test_acc = []

try:
    for i in range(n_epochs):
        train_loss = train_epoch(model, train_loader, criterion, optimizer, scheduler)  
        test_loss, test_acc = test_model(model, test_loader, criterion)
        
        Train_loss.append(train_loss)
        Test_loss.append(test_loss)
        Test_acc.append(test_acc)
        
        print('*' * 60)

except Exception as e:
    print("Error during training:", str(e))


Starting Training Epoch...
Train Loss: 5.27231628122464
Test Loss: 5.1876811730234245 Accuracy: 2.7182866556836904 %
************************************************************
Starting Training Epoch...
Train Loss: 5.16506778287216
Test Loss: 5.079177213342566 Accuracy: 4.757001647446458 %
************************************************************
Starting Training Epoch...
Train Loss: 5.07921332372746
Test Loss: 4.977169165485783 Accuracy: 6.3426688632619435 %
************************************************************
Starting Training Epoch...
Train Loss: 5.000325630080532


KeyboardInterrupt: 

In [22]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=1e-4, nesterov=True)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, factor=0.5, threshold=0.01)


In [24]:
n_epochs = 10
Train_loss = []
Test_loss = []
Test_acc = []

for i in range(n_epochs):
    print(f" Epoch {i+1}/{n_epochs} - Training 10 layers")

    train_loss = train_epoch(model, train_loader, criterion, optimizer, scheduler)
    test_loss, test_acc = test_model(model, test_loader, criterion)

    Train_loss.append(train_loss)
    Test_loss.append(test_loss)
    Test_acc.append(test_acc)

    scheduler.step(test_acc)  # עדכון ה-LR לפי הדיוק

    # 🔹 הדפסת ה-LR החדש אחרי כל אפוק
    print(f"Current Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")

    print('*' * 60)


 Epoch 1/10 - Training 10 layers
Starting Training Epoch...
Train Loss: 4.88957930820089
Test Loss: 4.642092246758311 Accuracy: 15.56836902800659 %
Current Learning Rate: 0.050000
************************************************************
 Epoch 2/10 - Training 10 layers
Starting Training Epoch...
Train Loss: 4.658906694868921
Test Loss: 4.395811375818755 Accuracy: 22.67298187808896 %
Current Learning Rate: 0.050000
************************************************************
 Epoch 3/10 - Training 10 layers
Starting Training Epoch...
Train Loss: 4.431320269007078
Test Loss: 4.16713064752127 Accuracy: 29.654036243822073 %
Current Learning Rate: 0.050000
************************************************************
 Epoch 4/10 - Training 10 layers
Starting Training Epoch...
Train Loss: 4.19763451629961
Test Loss: 3.884913610784631 Accuracy: 31.960461285008236 %
Current Learning Rate: 0.050000
************************************************************
 Epoch 5/10 - Training 10 layers


In [26]:
# המשך אפוקים
n_more_epochs = 10

for i in range(n_more_epochs):
    print(f" Epoch {i+1}/{n_more_epochs} - Continued Training")

    train_loss = train_epoch(model, train_loader, criterion, optimizer, scheduler)
    test_loss, test_acc = test_model(model, test_loader, criterion)

    Train_loss.append(train_loss)
    Test_loss.append(test_loss)
    Test_acc.append(test_acc)

    scheduler.step(test_acc)
    print(f"Current Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
    print('*' * 60)


 Epoch 1/10 - Continued Training
Starting Training Epoch...
Train Loss: 2.1615044449416683
Test Loss: 2.553974782165728 Accuracy: 55.621911037891266 %
Current Learning Rate: 0.050000
************************************************************
 Epoch 2/10 - Continued Training
Starting Training Epoch...
Train Loss: 1.8698703634906824
Test Loss: 2.475445583462715 Accuracy: 55.93080724876442 %
Current Learning Rate: 0.050000
************************************************************
 Epoch 3/10 - Continued Training
Starting Training Epoch...


KeyboardInterrupt: 

In [28]:

for param_group in optimizer.param_groups:
    param_group['lr'] = 0.005  

print(f" Learning Rate עודכן ל: {optimizer.param_groups[0]['lr']}")

 Learning Rate עודכן ל: 0.005


In [32]:
continued_epochs = 5

for i in range(continued_epochs):
    print(f" Epoch {i+1}/{continued_epochs} - Fine-tuning עם LR נמוך")

    train_loss = train_epoch(model, train_loader, criterion, optimizer, scheduler)
    test_loss, test_acc = test_model(model, test_loader, criterion)

    Train_loss.append(train_loss)
    Test_loss.append(test_loss)
    Test_acc.append(test_acc)

    scheduler.step(test_acc)
    print(f"Current Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
    print("*" * 60)

 Epoch 1/5 - Fine-tuning עם LR נמוך
Starting Training Epoch...
Train Loss: 1.1933471820723842
Test Loss: 2.195659223355745 Accuracy: 68.1836902800659 %
Current Learning Rate: 0.005000
************************************************************
 Epoch 2/5 - Fine-tuning עם LR נמוך
Starting Training Epoch...
Train Loss: 1.0357932611250542
Test Loss: 2.218760327288979 Accuracy: 68.55436573311367 %
Current Learning Rate: 0.005000
************************************************************
 Epoch 3/5 - Fine-tuning עם LR נמוך
Starting Training Epoch...
Train Loss: 0.9454093690489379
Test Loss: 2.2059662083261893 Accuracy: 68.49258649093905 %
Current Learning Rate: 0.005000
************************************************************
 Epoch 4/5 - Fine-tuning עם LR נמוך
Starting Training Epoch...
Train Loss: 0.8852810681705744
Test Loss: 2.2078153836099723 Accuracy: 69.83113673805602 %
Current Learning Rate: 0.005000
************************************************************
 Epoch 5/5 - Fi

In [34]:
# שמירה מלאה של המודל
torch.save(model, "resnet50_cars196_full_model.pth")
print("Model saved successfully.")


Model saved successfully.


In [36]:
# חלוקה חדשה לדאטה
dataset_size = len(cars_dataset)
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size

train_indices, test_indices = train_test_split(
    range(dataset_size),
    train_size=train_size,
    test_size=test_size,
    random_state=42
)

# עטיפה עם הטרנספורמציות
train_dataset = CarsDatasetWrapper(Subset(cars_dataset, train_indices), transform=train_transform)
test_dataset = CarsDatasetWrapper(Subset(cars_dataset, test_indices), transform=test_transform)

#  DataLoader חדשים
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

print(f" Training Samples: {len(train_dataset)}, Test Samples: {len(test_dataset)}")


 Training Samples: 12948, Test Samples: 3237


In [ ]:
# לוודא שהמודל במצב הערכה
model.eval()

# הרצת הבדיקה
test_loss, test_acc = test_model(model, test_loader, criterion)

# הדפסת התוצאות
print("Evaluation Results on New Test Set (80/20 split):")
print(f"Test Loss: {test_loss:.4f} | Accuracy: {test_acc:.2f}%")
print("*" * 60)


In [ ]:
##########